In [ ]:
import pandas as pd
import numpy as np
import os 
#!pip install plotly
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"


def create_major_level_trigger(df,pip_decimals = 2, level_pips = 50):
    def digit_finder(number1,number2,pip_decimals = 2, level_pips = 50):
        number1 = str(number1)
        decimal_loc1 = number1.find('.')
        first_digit1 = number1[decimal_loc1 + pip_decimals - 1]

        number2 = str(number2)
        decimal_loc2 = number2.find('.')
        first_digit2 = number2[decimal_loc2 + pip_decimals - 1]


        hunna1 = number1[decimal_loc1 + pip_decimals - 2]
        if hunna1 == '.':
            hunna1 = number1[decimal_loc1 + pip_decimals - 3 ]
        hunna2 = number2[decimal_loc2 + pip_decimals - 2]
        if hunna2 == '.':
            hunna2 = number2[decimal_loc2 + pip_decimals - 3 ]
        hunna1 = int(hunna1)
        hunna2 = int(hunna2)
        #print('NUMBER 1:',number1,decimal_loc1,first_digit1,hunna1)
        #print('NUMBER 2:',number2,decimal_loc2,first_digit2,hunna2)  

        checker = 0
        support = 0
        if level_pips == 50:
            pip_compare1 = int(first_digit1 + '0')
            pip_compare2 = int(first_digit2 + '0')
            if hunna1 > hunna2:
                checker = 1
                #print('100')
                support = 1
            elif hunna1 < hunna2:
                checker = 1     
                #print('100')
            elif pip_compare1 < 50 and pip_compare2 >= 50:
                checker = 1
                #print('50')
            elif pip_compare1 >= 50 and pip_compare2 < 50:
                checker = 1
                support = 1
                #print('50')

        elif level_pips == 100:
            if hunna1 > hunna2:
                checker = 1
            elif hunna1 < hunna2:
                checker = 1

        else:
            raise Exception('ERROR: level pips must be 50 or 100...')

        return checker ,support
    
    """Loop through all values of the dataset and create a trigger for when it crosses a major level
    if there has been a recent trigger hit then don't do anything
    """
    
    o_col = df.columns.get_loc('o')
    h_col = df.columns.get_loc('h')
    l_col = df.columns.get_loc('l')
    df['major_level_trigger'] = 0
    arr = df.values
    for i in range(1,arr.shape[0]):
        # testing resistance
        trigger = 0
        number1 = arr[i - 1,o_col]
        number2 = arr[i,h_col]
        checker,support = digit_finder(number1,number2,pip_decimals = pip_decimals, level_pips = level_pips)
        if checker == 1:
            trigger = 1

        number2 = arr[i,l_col]
        checker,support = digit_finder(number1,number2,pip_decimals = pip_decimals, level_pips = level_pips)
        if checker == 1:
            trigger = 1
        
        if sum(arr[i - 40 : i,-1]) == 0:
            arr[i,-1] = trigger
    
    return pd.DataFrame(arr,columns = df.columns)
def load_data(cur,timeframe,start,end,level_pips,pip_decimals):
    print('LOADING DATASET',cur,timeframe,start,end)
    file = os.getcwd() + '/' + cur + '_' + timeframe + '/' + str(start) + '_' + str(end) + '.csv'
    print('file path',file)
    df = pd.read_csv(file)
    df = create_major_level_trigger(df,pip_decimals = pip_decimals, level_pips = level_pips)
    print(df.shape)
    return df
def print_example(df,ind_list,i = 30,delta = 240):
    i = ind_list[i]   
    print('row loc:',i)
    fig = go.Figure(data=go.Candlestick(x=df.iloc[i - delta:i + (delta ),:]['time'],
                        open=df.iloc[i - delta:i + (delta ),:]['o'],
                        high=df.iloc[i - delta:i + (delta ),:]['h'],
                        low=df.iloc[i - delta:i + (delta ),:]['l'],
                        close=df.iloc[i - delta:i + (delta ),:]['c']))

    fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                  annotation_text="trigger point" , annotation_position="top left",
                  fillcolor="green", opacity=0.25, line_width=0)
    


    print(fig.show())

print(os.getcwd())
cur = 'USD_JPY'
cur = 'EUR_USD'
cur = 'XAU_USD'
cur = 'USD_JPY'
timeframe = 'M15'
start = date(2019,1,1)
end = date(2022,1,31)

df = load_data(cur = cur,timeframe = timeframe,start = start,end = end, level_pips = 100,pip_decimals = 2)
#df = df.iloc[2000:,:]
#df.index = list(range(df.shape[0]))
ind_list = list(df[df['major_level_trigger'] == 1].index)
df.head()

In [ ]:
print('LOADING DATASET',cur,timeframe,start,end)
file = os.getcwd() + '/' + cur + '_' + timeframe + '/' + str(start) + '_' + str(end) + '.csv'
print('file path',file)
df = pd.read_csv(file)



In [155]:
from os import listdir
import os
from os.path import isfile, join
import pandas as pd
import numpy as np
import os 
#!pip install plotly
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"
folder_name = 'USD_JPY_S5'
folder_name = 'EUR_USD_S5'
folder_name = 'USD_JPY_S5'
mypath = os.getcwd() + '/' + folder_name
file_list = [f for f in listdir(mypath) if isfile(join(mypath, f))]
#print(file_list)

def load_data(folder_name,loop_range):

    mypath = os.getcwd() + '/' + folder_name
    file_list = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    file_list.sort()
    print(file_list)
    final_df = pd.DataFrame([0])
    for i in range(loop_range):
        #print('loop range:',i)

        print(os.getcwd() + '/' + folder_name + '/' + file_list[i])
        df = pd.read_csv(os.getcwd() + '/' + folder_name + '/' + file_list[i])
        print('loop range:',i,df.shape)    
        if final_df.shape[0] == 1:
            final_df = df
        else:
            final_df = final_df.append(df)
        print('FINAL DF SHAPE:',final_df.shape)
        print()
        print()
    return final_df #df.sort_values(by = 'delta',ascending = False)
def print_example(df,ind_list,i = 30,delta = 240):
    i = ind_list[i]   
    fig = go.Figure(data=go.Ohlc(x=df.iloc[i - delta:i + (delta * 2),:]['time'],
                        open=df.iloc[i - delta:i + (delta * 2),:]['o'],
                        high=df.iloc[i - delta:i + (delta * 2),:]['h'],
                        low=df.iloc[i - delta:i + (delta * 2),:]['l'],
                        close=df.iloc[i - delta:i + (delta * 2),:]['c']))

    fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                  annotation_text="trigger point" , annotation_position="top left",
                  fillcolor="green", opacity=0.25, line_width=0)
    


    print(fig.show())

def create_spike_trigger(df,col = 'delta_max',lookback_threshold = 15000):
    df[col + '_spike_trigger'] = 0
    col_loc = df.columns.get_loc(col)
    arr = df.values
    for i in range(lookback_threshold,arr.shape[0]):
        if i % 200000 == 0:
            print(i)
        try:
            cur_max = max(arr[i - lookback_threshold : i ,col_loc])
        except:
            print('ERROR',i)
            cur_max = 100
        if arr[i,col_loc] > cur_max and sum(arr[i - 100 : i,-1]) == 0:
            arr[i,-1] = 1
    return pd.DataFrame(arr,columns = df.columns)
folder_name = 'USD_JPY_S5'
folder_name = 'GBP_JPY_S5'
folder_name = 'EUR_USD_S5'
folder_name = 'GBP_JPY_S5'
folder_name = 'USD_JPY_S5'
folder_name = 'XAU_USD_S5'
folder_name = 'USD_JPY_S5'
file_ind = 0

#df = load_data(folder_name = folder_name,loop_range = 12)
#ind_list = list(df.sort_values(by = 'delta',ascending = False).index)
#df = create_spike_trigger(df,col = 'delta_max',lookback_threshold = 10000)
#print(df.shape)
#df.head()

In [ ]:
filename = os.getcwd() + '/' + folder_name + '/delta_max_spike_trigger'
df.to_csv(filename,index = False)

In [ ]:
def load_data(folder_name,loop_range):

    mypath = os.getcwd() + '/' + folder_name
    file_list = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    file_list.sort()
    print(file_list)
    final_df = pd.DataFrame([0])
    for i in range(loop_range):
        #print('loop range:',i)

        print(os.getcwd() + '/' + folder_name + '/' + file_list[i])
        df = pd.read_csv(os.getcwd() + '/' + folder_name + '/' + file_list[i])
        print('loop range:',i,df.shape)    
        if final_df.shape[0] == 1:
            final_df = df
        else:
            final_df = final_df.append(df)
        print('FINAL DF SHAPE:',final_df.shape)
        print()
        print()
    return final_df #df.sort_values(by = 'delta',ascending = False)
folder_name = 'XAU_USD_S5'
df = load_data(folder_name = folder_name,loop_range = 12)

# Load Data

In [156]:
filename = os.getcwd() + '/' + folder_name + '/delta_max_spike_trigger'
df = pd.read_csv(filename)
print(df.shape)
df.head()





(2960618, 32)


,time,volume,o,h,l,c,delta,delta_max,delta_vector,max_vector,...,next_10,next_candle+1,next_2+1,next_3+1,next_4+1,next_5+1,next_10+1,next_20+1,trigger,delta_max_spike_trigger
0,2020-12-01T05:00:00.000000000Z,5,104.374,104.378,104.374,104.378,0.004,0.004,0.004,0.004,...,0.011,0.000,0.003,0.004,0.004,0.006,0.010,0.021,0,0
1,2020-12-01T05:00:05.000000000Z,4,104.378,104.378,104.378,104.378,0.000,0.000,0.000,-0.000,...,0.010,0.003,0.004,0.004,0.006,0.008,0.011,0.019,0,0
2,2020-12-01T05:00:10.000000000Z,1,104.378,104.378,104.378,104.378,0.000,0.000,0.000,-0.000,...,0.011,0.001,0.001,0.003,0.005,0.009,0.009,0.011,0,0
3,2020-12-01T05:00:15.000000000Z,6,104.378,104.381,104.376,104.381,0.003,0.005,0.003,0.005,...,0.009,0.000,0.002,0.004,0.008,0.008,0.008,0.010,0,0
4,2020-12-01T05:00:20.000000000Z,19,104.380,104.384,104.380,104.382,0.002,0.004,0.002,0.004,...,0.008,0.002,0.004,0.008,0.008,0.007,0.011,0.011,0,0


# USD JPY

In [157]:
def print_example(df,ind_list,ind = 30,delta = 240):
    i = ind_list[ind]   
    print('row loc:',i,' ind:',ind)
    fig = go.Figure(data=go.Ohlc(x=df.iloc[i - delta:i + (delta * 2),:]['time'],
                        open=df.iloc[i - delta:i + (delta * 2),:]['o'],
                        high=df.iloc[i - delta:i + (delta * 2),:]['h'],
                        low=df.iloc[i - delta:i + (delta * 2),:]['l'],
                        close=df.iloc[i - delta:i + (delta * 2),:]['c']))

    fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                  annotation_text="trigger point" , annotation_position="top left",
                  fillcolor="green", opacity=0.25, line_width=0)
    


    print(fig.show())
ind_list = list(df[df['delta_max_spike_trigger'] == 1].index)
i = 0
print_example(df,ind_list,ind = i,delta = 240)
i += 1

row loc: 19835  ind: 0


None


In [158]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

row loc: 27561  ind: 1


None


In [159]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

row loc: 32962  ind: 2


None


In [160]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

row loc: 40197  ind: 3


None


In [164]:
i = 35
print_example(df,ind_list,ind = i,delta = 240)
i += 1

row loc: 427889  ind: 35


None


In [165]:

print_example(df,ind_list,ind = i,delta = 240)
i += 1

row loc: 439001  ind: 36


None


In [166]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

row loc: 452376  ind: 37


None


In [167]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

row loc: 463711  ind: 38


None


In [168]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

row loc: 475783  ind: 39


None


In [169]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

row loc: 481241  ind: 40


None


In [170]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

row loc: 488683  ind: 41


None


In [171]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

row loc: 499044  ind: 42


None


In [172]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

row loc: 499342  ind: 43


None


In [173]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

row loc: 510166  ind: 44


None


In [174]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

row loc: 529206  ind: 45


None


In [175]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

row loc: 546946  ind: 46


None


In [185]:
arr = df.values 
o = df.columns.get_loc('o')
c = df.columns.get_loc('c')
dv = df.columns.get_loc('delta_vector')
lst = []
cols = ['delta_vector',
        'next_5',
        'next_min',
                    'next_5_min',
                    'next_10_min',
                    'next_30_min',
                    'next_60_min',
                    'next_240_min'

       ]
col = 'delta_max_spike_trigger'
for i in range(arr.shape[0]):
    if arr[i,col] == 1:
        minute = (60 / 5)
        next_5 = arr[i + 6,c] - arr[i + 1,o]
        next_min = arr[i + minute,c] - arr[i + 1,o]
        next_5_min = arr[i + (minute * 5),c] - arr[i + 1,o]
        next_10_min = arr[i + (minute * 10),c] - arr[i + 1,o]
        next_30_min = arr[i + (minute * 30),c] - arr[i + 1,o]
        next_60_min = arr[i + (minute * 60),c] - arr[i + 1,o]
        next_240_min = arr[i + (minute * 240),c] - arr[i + 1,o]
        lst.append([arr[i,dv],
                    next_5,
                    next_min,
                    next_5_min,
                    next_10_min,
                    next_30_min,
                    next_60_min,
                    next_240_min
                   ]

                    )

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [186]:
i

0

In [176]:
df.head()

,time,volume,o,h,l,c,delta,delta_max,delta_vector,max_vector,...,next_10,next_candle+1,next_2+1,next_3+1,next_4+1,next_5+1,next_10+1,next_20+1,trigger,delta_max_spike_trigger
0,2020-12-01T05:00:00.000000000Z,5,104.374,104.378,104.374,104.378,0.004,0.004,0.004,0.004,...,0.011,0.000,0.003,0.004,0.004,0.006,0.010,0.021,0,0
1,2020-12-01T05:00:05.000000000Z,4,104.378,104.378,104.378,104.378,0.000,0.000,0.000,-0.000,...,0.010,0.003,0.004,0.004,0.006,0.008,0.011,0.019,0,0
2,2020-12-01T05:00:10.000000000Z,1,104.378,104.378,104.378,104.378,0.000,0.000,0.000,-0.000,...,0.011,0.001,0.001,0.003,0.005,0.009,0.009,0.011,0,0
3,2020-12-01T05:00:15.000000000Z,6,104.378,104.381,104.376,104.381,0.003,0.005,0.003,0.005,...,0.009,0.000,0.002,0.004,0.008,0.008,0.008,0.010,0,0
4,2020-12-01T05:00:20.000000000Z,19,104.380,104.384,104.380,104.382,0.002,0.004,0.002,0.004,...,0.008,0.002,0.004,0.008,0.008,0.007,0.011,0.011,0,0


In [216]:


def check_strategy_basic_stats(df,col_name = 'delta_max_spike_trigger'):
    arr = df.values 
    o = df.columns.get_loc('o')
    c = df.columns.get_loc('c')
    dv = df.columns.get_loc('delta_vector')
    lst = []
    cols = ['delta_vector',
            'next_5',
            'next_min',
                        'next_5_min',
                        'next_10_min',
                        'next_30_min',
                        'next_60_min',
                        'next_240_min',
            'next_480_min'
            
           ]
    col = df.columns.get_loc(col_name)
    print(col)
    minute = int(60 / 5)
    for i in range(arr.shape[0]):
        if arr[i,col] == 1:
            
            next_5 = arr[i + 6,c] - arr[i + 1,o]
            next_min = arr[i + minute,c] - arr[i + 1,o]
            next_5_min = arr[i + (minute * 5),c] - arr[i + 1,o]
            next_10_min = arr[i + (minute * 10),c] - arr[i + 1,o]
            next_30_min = arr[i + (minute * 30),c] - arr[i + 1,o]
            next_60_min = arr[i + (minute * 60),c] - arr[i + 1,o]
            next_240_min = arr[i + (minute * 240),c] - arr[i + 1,o]
            next_480_min = arr[i + (minute * 480),c] - arr[i + 1,o]
            lst.append([arr[i,dv],
                        next_5,
                        next_min,
                        next_5_min,
                        next_10_min,
                        next_30_min,
                        next_60_min,
                        next_240_min,
                        next_480_min
                       ]
                        
                        )
    return pd.DataFrame(lst,columns = cols)
            
            
        
        
test = check_strategy_basic_stats(df,col_name = 'delta_max_spike_trigger')
test.head()
    
    

31


,delta_vector,next_5,next_min,next_5_min,next_10_min,next_30_min,next_60_min,next_240_min,next_480_min
0,0.025,-0.013,-0.004,-0.016,-0.024,-0.056,-0.099,-0.107,-0.286
1,-0.038,-0.001,-0.003,-0.003,-0.035,-0.043,-0.089,-0.319,-0.307
2,-0.032,-0.003,-0.006,0.023,0.021,-0.003,-0.052,0.111,0.181
3,-0.052,-0.038,-0.040,0.009,-0.091,0.051,0.003,0.151,0.105
4,0.026,0.061,0.044,0.023,0.020,0.040,0.054,0.110,0.123


In [211]:
df.tail()

,time,volume,o,h,l,c,delta,delta_max,delta_vector,max_vector,...,next_10,next_candle+1,next_2+1,next_3+1,next_4+1,next_5+1,next_10+1,next_20+1,trigger,delta_max_spike_trigger
2960613,2021-12-01T04:59:20.000000000Z,10,113.496,113.504,113.494,113.502,0.006,0.010,0.006,0.010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2960614,2021-12-01T04:59:25.000000000Z,1,113.501,113.501,113.501,113.501,0.000,0.000,0.000,-0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2960615,2021-12-01T04:59:30.000000000Z,1,113.500,113.500,113.500,113.500,0.000,0.000,0.000,-0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2960616,2021-12-01T04:59:45.000000000Z,11,113.498,113.499,113.498,113.498,0.000,0.001,0.000,-0.001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2960617,2021-12-01T04:59:55.000000000Z,5,113.499,113.499,113.498,113.499,0.000,0.001,0.000,-0.001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [221]:
def compound_interest(principle, rate, time):
 
    # Calculates compound interest
    Amount = principle * (pow((1 + rate / 100), time))
    CI = Amount - principle
    print("Compound interest is", CI)
    return Amount

principle = 10000
rate = 1
time = 100
amt = compound_interest(principle, rate, time)
amt

Compound interest is 17048.138294215285


27048.138294215285

In [213]:
pos = test[test['delta_vector'] > 0]
neg = test[test['delta_vector'] < 0]
print('POS SUM')
print(pos.sum())
print('POS MEAN')
print(pos.mean())
print()
print('NEG SUM')
print(neg.sum())
print('NEG MEAN')
print(neg.mean())

POS SUM
delta_vector    6.132
next_5         -1.160
next_min       -1.392
next_5_min     -1.297
next_10_min    -0.919
next_30_min    -0.109
next_60_min    -0.285
next_240_min    3.038
next_480_min    3.126
dtype: float64
POS MEAN
delta_vector    0.041154
next_5         -0.007785
next_min       -0.009342
next_5_min     -0.008705
next_10_min    -0.006168
next_30_min    -0.000732
next_60_min    -0.001913
next_240_min    0.020389
next_480_min    0.020980
dtype: float64

NEG SUM
delta_vector   -5.289
next_5         -0.553
next_min       -0.558
next_5_min     -0.026
next_10_min     0.827
next_30_min     0.824
next_60_min     0.002
next_240_min    2.983
next_480_min    0.149
dtype: float64
NEG MEAN
delta_vector   -0.041320
next_5         -0.004320
next_min       -0.004359
next_5_min     -0.000203
next_10_min     0.006461
next_30_min     0.006437
next_60_min     0.000016
next_240_min    0.023305
next_480_min    0.001164
dtype: float64


In [214]:
pos.sort_values(by = 'next_240_min',ascending = False).head(40)

,delta_vector,next_5,next_min,next_5_min,next_10_min,next_30_min,next_60_min,next_240_min,next_480_min
148,0.055,0.136,0.252,0.366,0.242,0.338,0.479,0.761,0.669
268,0.066,0.060,0.063,0.253,0.272,0.444,0.444,0.624,0.830
129,0.042,-0.036,-0.032,-0.006,-0.014,0.148,0.114,0.589,0.607
118,0.258,-0.013,0.021,0.129,0.132,-0.026,0.126,0.571,0.649
257,0.164,0.027,0.070,0.002,0.014,0.056,0.008,0.562,0.590
130,0.012,-0.040,-0.020,-0.056,-0.026,0.013,0.094,0.510,0.564
274,0.082,-0.030,-0.062,0.020,0.008,0.181,0.210,0.483,0.041
222,0.021,0.006,0.016,0.020,0.028,0.082,0.074,0.463,0.398
136,0.064,0.012,0.003,0.002,0.060,0.035,0.128,0.415,0.294
82,0.028,-0.002,0.014,0.008,0.004,0.062,0.057,0.384,0.300


In [198]:

col_name = 'delta_max_spike_trigger'
arr = df.values 
o = df.columns.get_loc('o')
c = df.columns.get_loc('c')
dv = df.columns.get_loc('delta_vector')
lst = []
cols = ['delta_vector',
        'next_5',
        'next_min',
        'next_5_min',
        'next_10_min',
        'next_30_min',
        'next_60_min',
        'next_240_min'

       ]
col = df.columns.get_loc(col_name)
print(col)
for i in range(arr.shape[0]):
    if arr[i,col] == 1:
        minute = int(60 / 5)
        next_5 = arr[i + 6,c] - arr[i + 1,o]
        next_min = arr[i + minute,c] - arr[i + 1,o]
        next_5_min = arr[i + (minute * 5),c] - arr[i + 1,o]
        next_10_min = arr[i + (minute * 10),c] - arr[i + 1,o]
        next_30_min = arr[i + (minute * 30),c] - arr[i + 1,o]
        next_60_min = arr[i + (minute * 60),c] - arr[i + 1,o]
        next_240_min = arr[i + (minute * 240),c] - arr[i + 1,o]
        lst.append([arr[i,dv],
                    next_5,
                    next_min,
                    next_5_min,
                    next_10_min,
                    next_30_min,
                    next_60_min,
                    next_240_min
                   ]

                    )
        
        
        

31


In [194]:
minute

12.0

In [195]:
c

5

In [196]:
o

2

In [197]:
i + minute

19847.0

In [179]:
col = 'delta_max_spike_trigger'
df[df[col] == 1].shape

(279, 32)

In [181]:
(60 / 5) * 30

360.0

In [183]:
uno = (60 / 5)
uno * 5

60.0

# XAU USD

In [ ]:
def print_example(df,ind_list,ind = 30,delta = 240):
    i = ind_list[ind]   
    print('row loc:',i,' ind:',ind)
    fig = go.Figure(data=go.Ohlc(x=df.iloc[i - delta:i + (delta * 2),:]['time'],
                        open=df.iloc[i - delta:i + (delta * 2),:]['o'],
                        high=df.iloc[i - delta:i + (delta * 2),:]['h'],
                        low=df.iloc[i - delta:i + (delta * 2),:]['l'],
                        close=df.iloc[i - delta:i + (delta * 2),:]['c']))

    fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                  annotation_text="trigger point" , annotation_position="top left",
                  fillcolor="green", opacity=0.25, line_width=0)
    


    print(fig.show())

ind_list = list(df[df['delta_max_spike_trigger'] == 1].index)
i = 0
print_example(df,ind_list,ind = i,delta = 240)
i += 1



In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1


In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
df.head()

In [ ]:

from datetime import datetime, date, timedelta
import time 
def consecutive_candles(df):
    print('getting consecutive candles...')
    s = time.time()
    df['consecutive_candles'] = 1
    col = 'delta_vector'
    col_loc = df.columns.get_loc('delta_vector')
    arr = df.values 
    
    for i in range(arr.shape[0]):
        tester = True
        starting_val = arr[i - 1,col_loc]
        if starting_val >= 0:
            pos = True
        else:
            pos = False
        counter = 1
        while tester:
            val = arr[i - 1 - counter,col_loc]
            if val >= 0:
                pos2 = True
            else:
                pos2 = False     
            
            if pos == pos2:
                counter += 1
            else:
                tester = False
        arr[i,-1] = counter
    e = time.time()
    print('TOTAL TIME:',(e - s)/60,' MINUTES')
    return pd.DataFrame(arr,columns = df.columns)
            
            
test = consecutive_candles(df)
print(test.shape)
test.head()

In [ ]:
test.consecutive_candles.max()

In [ ]:
test.iloc[20:30,:]

# USD JPY

In [ ]:
def print_example(df,ind_list,ind = 30,delta = 240):
    i = ind_list[ind]   
    print('row loc:',i,' ind:',ind)
    fig = go.Figure(data=go.Ohlc(x=df.iloc[i - delta:i + (delta * 2),:]['time'],
                        open=df.iloc[i - delta:i + (delta * 2),:]['o'],
                        high=df.iloc[i - delta:i + (delta * 2),:]['h'],
                        low=df.iloc[i - delta:i + (delta * 2),:]['l'],
                        close=df.iloc[i - delta:i + (delta * 2),:]['c']))

    fig.add_vrect(x0=df['time'].iloc[i - 1], x1=df['time'].iloc[i+1], 
                  annotation_text="trigger point" , annotation_position="top left",
                  fillcolor="green", opacity=0.25, line_width=0)
    


    print(fig.show())

ind_list = list(df[df['delta_max_spike_trigger'] == 1].index)
i = 0
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
i = 30
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 400)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

# GBP JPY

In [ ]:
ind_list = list(df[df['delta_max_spike_trigger'] == 1].index)
i = 0
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1

In [ ]:
print_example(df,ind_list,ind = i,delta = 240)
i += 1